## Score challenge submission

This notebook produces scores for a custom challenge set.  It scores each tasks and provides a summary score (average).

This is designed to help compare the quality of a solution against a challenge set under our control and demonstrate that the aicrowd score is consistent.

In [ ]:
import sys
import json
import re
import collections
import os
import datetime
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import time
import pickle

In [ ]:
challenge="ex2-from-21k-train-with-pids"
submitdir="/home/jpr/projects/mpd-challenge-aicrowd/"

submissionfile="method-01-mympd-2nd-21k-2021-11-14.csv.gz"
submissions = [{"tag": "vl6", "trainsizek": 21, "dir": submitdir, "file": submissionfile}]

submissionfile="method-02-mympd-2nd-21k-2021-11-07.csv.gz"
submissions.append({"tag": "hw", "trainsizek": 21, "dir": submitdir, "file": submissionfile})

submissionfile="method-u2uknn-mympd-full-mympd-full-20k-2021-12-30.csv.gz"
submissions.append({"method": "u2uknn", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-20k", 
                    "trainsizek": 20, 
                    "tag": "u2uknnfull-20k",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-40k-2021-12-31.csv.gz"
submissions.append({"method": "u2uknn", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-40k", 
                    "trainsizek": 40, 
                    "tag": "u2uknnfull-40k", 
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-60k-2021-12-31.csv.gz"
submissions.append({"method": "u2uknn", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-60k", 
                    "trainsizek": 60, 
                    "tag": "u2uknnfull-60k", 
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-80k-2021-12-31.csv.gz"
submissions.append({"method": "u2uknn", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-80k", 
                    "trainsizek": 80, 
                    "tag": "u2uknnfull-80k", 
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-100k-2021-12-31.csv.gz"
submissions.append({"method": "u2uknn", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-100k", 
                    "trainsizek": 100, 
                    "tag": "u2uknnfull-100k", 
                    "dir": submitdir,
                    "file": submissionfile})

submitdir="/home/jpr/projects/mpd-challenge-aicrowd/2022-01-02-scaling/"

submissionfile="method-u2uknn-mympd-full-mympd-full-20k-2022-01-07.csv.gz"
submissions.append({"method": "u2uknn", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-20k", 
                    "trainsizek": 20, 
                    "tag": "u2uknnfull-20k-v2",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-200k-2022-01-07.csv.gz"
submissions.append({"method": "u2uknn", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-200k", 
                    "trainsizek": 200, 
                    "tag": "u2uknnfull-200k-v2",
                    "dir": submitdir,
                    "file": submissionfile})


submissionfile="method-u2uknn-mympd-full-mympd-full-300k-2022-01-08_06:46:35.csv.gz"
submissions.append({"method": "u2uknn", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-300k", 
                    "trainsizek": 300, 
                    "tag": "u2uknnfull-300k-v2",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-400k-2022-01-08_06:46:35.csv.gz"
submissions.append({"method": "u2uknn", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-400k", 
                    "trainsizek": 400, 
                    "tag": "u2uknnfull-400k-v2",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-500k-2022-01-08_09:46:39.csv.gz"
submissions.append({"method": "u2uknn", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-500k", 
                    "trainsizek": 500, 
                    "tag": "u2uknnfull-500k-v2",
                    "dir": submitdir,
                    "file": submissionfile})

submissionfile="method-u2uknn-mympd-full-mympd-full-20k-2022-01-12_19:32:21.csv.gz"
submissions.append({"method": "u2uknn-20220112", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-20k", 
                    "trainsizek": 20, 
                    "tag": "u2uknnfull-20k-v3",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-40k-2022-01-12_19:32:20.csv.gz"
submissions.append({"method": "u2uknn-20220112", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-40k", 
                    "trainsizek": 40, 
                    "tag": "u2uknnfull-40k-v3",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-60k-2022-01-12_19:32:20.csv.gz"
submissions.append({"method": "u2uknn-20220112", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-60k", 
                    "trainsizek": 60, 
                    "tag": "u2uknnfull-60k-v3",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-80k-2022-01-12_19:32:20.csv.gz"
submissions.append({"method": "u2uknn-20220112", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-80k", 
                    "trainsizek": 80, 
                    "tag": "u2uknnfull-80k-v3",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-100k-2022-01-12_19:32:20.csv.gz"
submissions.append({"method": "u2uknn-20220112", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-100k", 
                    "trainsizek": 100, 
                    "tag": "u2uknnfull-100k-v3",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-200k-2022-01-12_19:32:20.csv.gz"
submissions.append({"method": "u2uknn-20220112", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-200k", 
                    "trainsizek": 200, 
                    "tag": "u2uknnfull-200k-v3",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-300k-2022-01-12_19:32:20.csv.gz"
submissions.append({"method": "u2uknn-20220112", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-300k", 
                    "trainsizek": 300, 
                    "tag": "u2uknnfull-300k-v3",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-400k-2022-01-12_19:39:21.csv.gz"
submissions.append({"method": "u2uknn-20220112", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-400k", 
                    "trainsizek": 400, 
                    "tag": "u2uknnfull-400k-v3",
                    "dir": submitdir,
                    "file": submissionfile})


#submissionfile="method-u2uknn-mympd-full-mympd-full-500k-2022-01-12_19:43:06.csv.gz"
#submissions.append({"method": "u2uknn-20220112", 
#                    "challenge":"mympd-full", 
#                    "trainset":"mympd-full-500k", 
#                    "trainsizek": 500, 
#                    "tag": "u2uknnfull-500k-v3",
#                    "dir": submitdir,
#                    "file": submissionfile})

submissionfile="method-u2uknn-mympd-full-mympd-full-20k-2022-01-15_08:53:05.csv.gz"
submissions.append({"method": "u2uknn-20220115", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-20k", 
                    "trainsizek": 20, 
                    "tag": "u2uknnfull-20k-v2.0.1",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-20k-2022-01-16_13:20:36.csv.gz"
submissions.append({"method": "u2uknn-20220116", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-20k", 
                    "trainsizek": 20, 
                    "tag": "u2uknnfull-20k-v2.0.2",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-20k-2022-01-16_14:15:28.csv.gz"
submissions.append({"method": "u2uknn-20220116", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-20k", 
                    "trainsizek": 20, 
                    "tag": "u2uknnfull-20k-v2.1.0",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-20k-2022-01-17_09:59:27.csv.gz"
submissions.append({"method": "u2uknn-20220117", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-20k", 
                    "trainsizek": 20, 
                    "tag": "u2uknnfull-20k-v2.1.1",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-20k-2022-01-17_11:35:26.csv.gz"
submissions.append({"method": "u2uknn-20220117", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-20k", 
                    "trainsizek": 20, 
                    "tag": "u2uknnfull-20k-v2.2.0",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-20k-2022-01-17_12:13:32.csv.gz"
submissions.append({"method": "u2uknn-20220117", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-20k", 
                    "trainsizek": 20, 
                    "tag": "u2uknnfull-20k-v2.2.1",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-20k-2022-01-17_14:53:39.csv.gz"
submissions.append({"method": "u2uknn-20220117", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-20k", 
                    "trainsizek": 20, 
                    "tag": "u2uknnfull-20k-v2.2.2",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-20k-2022-01-17_15:33:31.csv.gz"
submissions.append({"method": "u2uknn-20220117", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-20k", 
                    "trainsizek": 20, 
                    "tag": "u2uknnfull-20k-v2.2.3",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-20k-2022-01-18_03:16:20.csv.gz"
submissions.append({"method": "u2uknn-20220118", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-20k", 
                    "trainsizek": 20, 
                    "tag": "u2uknnfull-20k-v2.2.4",
                    "dir": submitdir,
                    "file": submissionfile})


submissionfile="method-u2uknn-mympd-full-mympd-full-500k-2022-01-16_10:37:48.csv.gz"
submissions.append({"method": "u2uknn-20220116", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-500k", 
                    "trainsizek": 500, 
                    "tag": "u2uknnfull-500k-v2.0.1",
                    "dir": submitdir,
                    "file": submissionfile})


submissionfile="method-u2uknn-mympd-full-mympd-full-600k-2022-01-12_06:35:12.csv.gz"
submissions.append({"method": "u2uknn-20220112", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-600k", 
                    "trainsizek": 600, 
                    "tag": "u2uknnfull-600k-v3",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-700k-2022-01-12_06:35:12.csv.gz"
submissions.append({"method": "u2uknn-20220112", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-700k", 
                    "trainsizek": 700, 
                    "tag": "u2uknnfull-700k-v3",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-800k-2022-01-12_06:35:12.csv.gz"
submissions.append({"method": "u2uknn-20220112", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-800k", 
                    "trainsizek": 800, 
                    "tag": "u2uknnfull-800k-v3",
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-u2uknn-mympd-full-mympd-full-900k-2022-01-12_06:35:12.csv.gz"
submissions.append({"method": "u2uknn-20220112", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-900k", 
                    "trainsizek": 900, 
                    "tag": "u2uknnfull-900k-v3",
                    "dir": submitdir,
                    "file": submissionfile})


submissionfile="method-i2iknn-mympd-full-mympd-full-20k-2022-01-02.csv.gz"
submissions.append({"method": "i2iknn", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-20k",
                    "trainsizek": 20, 
                    "tag": "i2iknnfull-20k", 
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-i2iknn-mympd-full-mympd-full-40k-2022-01-02.csv.gz"
submissions.append({"method": "i2iknn", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-40k", 
                    "trainsizek": 40, 
                    "tag": "i2iknnfull-40k", 
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-i2iknn-mympd-full-mympd-full-60k-2022-01-02.csv.gz"
submissions.append({"method": "i2iknn", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-60k", 
                    "trainsizek": 60, 
                    "tag": "i2iknnfull-60k", 
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-i2iknn-mympd-full-mympd-full-80k-2022-01-02.csv.gz"
submissions.append({"method": "i2iknn", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-80k", 
                    "trainsizek": 80, 
                    "tag": "i2iknnfull-80k", 
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-i2iknn-mympd-full-mympd-full-100k-2022-01-02.csv.gz"
submissions.append({"method": "i2iknn", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-100k", 
                    "trainsizek": 100, 
                    "tag": "i2iknnfull-100k", 
                    "dir": submitdir,
                    "file": submissionfile})

In [ ]:
submitdir="/home/jpr/projects/mpd-challenge-aicrowd/2022-01-02-scaling/"
submissionfile="method-mfals-mympd-full-mympd-full-20k-2022-01-02.csv.gz"
submissions.append({"method": "mfals", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-20k", 
                    "trainsizek": 20, 
                    "tag": "mfalsfull-20k", 
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-mfals-mympd-full-mympd-full-40k-2022-01-02.csv.gz"
submissions.append({"method": "mfals", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-40k", 
                    "trainsizek": 40, 
                    "tag": "mfalsfull-40k", 
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-mfals-mympd-full-mympd-full-60k-2022-01-02.csv.gz"
submissions.append({"method": "mfals", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-60k", 
                    "trainsizek": 60, 
                    "tag": "mfalsfull-60k", 
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-mfals-mympd-full-mympd-full-80k-2022-01-02.csv.gz"
submissions.append({"method": "mfals", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-80k", 
                    "trainsizek": 80, 
                    "tag": "mfalsfull-80k", 
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-mfals-mympd-full-mympd-full-100k-2022-01-02.csv.gz"
submissions.append({"method": "mfals", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-100k", 
                    "trainsizek": 100, 
                    "tag": "mfalsfull-100k", 
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-mfals-mympd-full-mympd-full-200k-2022-01-06.csv.gz"
submissions.append({"method": "mfals", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-200k", 
                    "trainsizek": 200, 
                    "tag": "mfalsfull-200k", 
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-mfals-mympd-full-mympd-full-300k-2022-01-06.csv.gz"
submissions.append({"method": "mfals", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-300k", 
                    "trainsizek": 300, 
                    "tag": "mfalsfull-300k", 
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-mfals-mympd-full-mympd-full-400k-2022-01-06.csv.gz"
submissions.append({"method": "mfals", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-400k", 
                    "trainsizek": 400, 
                    "tag": "mfalsfull-400k", 
                    "dir": submitdir,
                    "file": submissionfile})
submissionfile="method-mfals-mympd-full-mympd-full-500k-2022-01-06.csv.gz"
submissions.append({"method": "mfals", 
                    "challenge":"mympd-full", 
                    "trainset":"mympd-full-500k", 
                    "trainsizek": 500, 
                    "tag": "mfalsfull-500k", 
                    "dir": submitdir,
                    "file": submissionfile})

In [ ]:
submitdir="/home/jpr/projects/mpd-challenge-aicrowd/"

submissionfile="method-u2uknn-mympd-mpd-2nd-21k-2021-12-12.csv.gz"
submissions.append({"tag": "u2uknn", "trainsizek": 21, "dir": submitdir, "file": submissionfile})

submissionfile="method-i2iknn-mympd-mpd-2nd-21k-2021-12-18.csv.gz"
submissions.append({"tag": "i2iknn", "trainsizek": 21, "dir": submitdir, "file": submissionfile})

submissionfile="method-mfals-mympd-mpd-2nd-21k-2021-12-26.csv.gz"
submissions.append({"tag": "mfals", "trainsizek": 21, "dir": submitdir, "file": submissionfile})

In [ ]:
submissions=[]

Experiments to find performance defecit of unified u2u/i2i knn notebook

In [ ]:
experiments=list()
#experiments.append("/home/jpr/projects/mpd-test-sets/results/i2i-scale-test")
#experiments.append("/home/jpr/projects/mpd-test-sets/results/i2i-direct-dotprod")
experiments.append("/home/jpr/projects/mpd-test-sets/results/i2i-original-sim")
experiments.append("/home/jpr/projects/mpd-test-sets/results/i2i-fast-sim")
experiments.append("/home/jpr/projects/mpd-test-sets/results/i2i-fast-sim-score-batch")
experiments.append("/home/jpr/projects/mpd-test-sets/results/i2i-orig-test")
experiments.append("/home/jpr/projects/mpd-test-sets/results/i2i-orig-in-unified-u2u")
experiments.append("/home/jpr/projects/mpd-test-sets/results/i2i-old-knn-i2i")
experiments.append("/home/jpr/projects/mpd-test-sets/results/i2i-orig-in-unified-u2u-score-truncate")
experiments.append("/home/jpr/projects/mpd-test-sets/results/i2i-orig-in-unified-u2u-score-c2a-batch")
experiments.append("/home/jpr/projects/mpd-test-sets/results/i2i-orig-in-unified-u2u-score-c2a-direct")
experiments.append("/home/jpr/projects/mpd-test-sets/results/i2i-orig-in-unified-u2u-score-c2a-batch-trainset-sim")

In [ ]:
experiments=list()
experiments.append("/home/jpr/projects/mpd-test-sets/results/u2u-optimized-scaled-complete")
experiments.append("/home/jpr/projects/mpd-test-sets/results/i2i-optimized-scaled-complete")
#experiments.append("/home/jpr/projects/mpd-test-sets/results/u2u-optimized-scaled-complete-redux")
#experiments.append("/home/jpr/projects/mpd-test-sets/results/mfals-manual-results-2022-01-02")

In [ ]:
#submissions=[]

for submitdir in experiments:
    with os.scandir(submitdir) as entries:
        for entry in entries:
            if entry.name.endswith(".gz"):
                print(entry.name)
                method, challenge, trainset, ignore = entry.name.split("_", 3)
                ignore, method = method.split("-", 1)
                trainset, size = trainset.rsplit("-", 1)
                size = size.split('k')[0]
                ignore, tag = submitdir.rsplit("/", 1)

                submissions.append({"method": method, 
                        "challenge": challenge, 
                        "trainset": trainset, 
                        "trainsize": size, 
                        "tag": tag, 
                        "dir": submitdir + "/",
                        "file": entry.name})
            

In [ ]:
submissions

In [ ]:
submitdir="/home/jpr/projects/mpd-challenge-aicrowd/"

In [ ]:
result_files={"vl6": "method-01-mympd-2nd-21k-2021-11-14.csv.gz",
        "hw": "method-02-mympd-2nd-21k-2021-11-07.csv.gz",
        "u2uknn": "method-u2uknn-mympd-mpd-2nd-21k-2021-12-12.csv.gz"}

### Load the no holdouts challenge set

In [ ]:
with open('data/{}/challenge_set_noholdout.json'.format(challenge),'r') as f:
    data = json.loads(f.read())

In [ ]:
noholdout = pd.json_normalize(data,"playlists")

In [ ]:
noholdout.head()

Reorder noholdouts to group sequential and random tasks.

In [ ]:
tmpdf=noholdout[0:6000]

In [ ]:
tmpdf=tmpdf.append(noholdout[7000:8000])

In [ ]:
tmpdf=tmpdf.append(noholdout[6000:7000])

In [ ]:
tmpdf=tmpdf.append(noholdout[8000:10000])

In [ ]:
tmpdf.task_name.unique()

In [ ]:
noholdout=tmpdf

In [ ]:
with open('data/{}/challenge_set_noholdout.json'.format("mympd-full"),'r') as f:
    data = json.loads(f.read())
    
 
noholdout2 = pd.json_normalize(data,"playlists")

In [ ]:
def reorder_holdout(noholdout):
    tmpdf=noholdout[0:6000]
    tmpdf=tmpdf.append(noholdout[7000:8000])
    tmpdf=tmpdf.append(noholdout[6000:7000])
    tmpdf=tmpdf.append(noholdout[8000:10000])
    return tmpdf

In [ ]:
noholdout2=reorder_holdout(noholdout2)

### Load challenge submission`

In [ ]:
rectracks=pd.read_csv('{}/{}'.format(submitdir, submissionfile), header=None, skiprows=1, index_col=0, skipinitialspace=True)


Make sure the pids match those of the challenge set.  In the case of mympd the pid range starts at 200000 and goes up in sequence of noholdout data set.

In [ ]:
rectracks.head()

In [ ]:
rectracks = rectracks.sort_values(by=0, axis='index')

In [ ]:
rectracks.head()

## Score Challenge Submission

In [ ]:
startpid=2000000

R-precision is the fraction of correctly recommended tracks in the ground truth playlist as described on [the challenge site](https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge#evaluation)
    

Use the [r-precesion calculation from the hello_world metrics](https://github.com/jprorama/spotify_recSys_challenge_2018/blob/f33d82715190a20fdbc998c9ff709bcabd62a55e/utils/metrics.py#L26)

In [ ]:
def get_r_precision(answer, cand):
    set_answer = set(answer)
    r = len(set_answer&set(cand[:len(set_answer)])) / len(set_answer)
    return r

def get_ndcg(answer, cand):
    cand_len = len(cand) 
    idcg=0
    dcg=0
    
    #print("cand len {}".format(cand_len))
    #print("ans len {}".format(len(answer)))
    #print("cand: {}".format(cand))
    
    for i in range(cand_len):
        #print("i {}".format(i))
        #print("cand {}".format(cand[i]))
        if cand[i] in answer: 
            dcg += (1/math.log(i+1+1,2))

    for i in range(len(set(answer))):
        idcg += (1/math.log(i+1+1,2))
    
    return dcg/idcg

In [ ]:
realtracks=pd.DataFrame()
rprec_sum = 0.0
ndcg_sum = 0.0
rprec_match = 0
ndcg_match = 0

for i in range(len(noholdout)):
    pid = startpid + i
    #print("pid={}".format(pid))
    gttracks = [track["track_uri"] for track in noholdout[noholdout["challenge_pid"]==pid].tracks.to_list()[0]]
    candtracks = rectracks.loc[pid].to_list()
    rprec = get_r_precision(gttracks, candtracks)
    rprec_sum = rprec_sum + rprec
    if rprec > 0:
        rprec_match += 1
    ndcg = get_ndcg(gttracks, candtracks)
    ndcg_sum = ndcg_sum + ndcg
    if ndcg > 0:
        ndcg_match += 1
    
print("rprec = {}".format(rprec_sum/len(noholdout)))
print("ndcg = {}".format(ndcg_sum/len(noholdout)))
print("rprec_match = {}".format(rprec_match))
print("ndcg_match = {}".format(ndcg_match))

In [ ]:
def score_set(answer, candidates):
    rprec_sum = 0.0
    ndcg_sum = 0.0
    rprec_match = 0
    ndcg_match = 0

    scores = dict()

    for pid in answer.challenge_pid:
        #pid = startpid + i
        #print("pid={}".format(pid))
        gttracks = [track["track_uri"] for track in answer[answer["challenge_pid"]==pid].tracks.to_list()[0]]
        candtracks = candidates.loc[pid].to_list()
        rprec = get_r_precision(gttracks, candtracks)
        rprec_sum = rprec_sum + rprec
        if rprec > 0:
            rprec_match += 1
        ndcg = get_ndcg(gttracks, candtracks)
        ndcg_sum = ndcg_sum + ndcg
        if ndcg > 0:
            ndcg_match += 1
        
    scores["rprec"] = rprec_sum/len(answer)
    scores["rprec_match"] = rprec_match
    scores["ndcg"] = ndcg_sum/len(answer)
    scores["ndcg_match"] = ndcg_match
        
    return scores

In [ ]:
rprec_sum = 0.0
ndcg_sum = 0.0
rprec_match = 0
ndcg_match = 0

scores = dict()

for task in noholdout.task_name.drop_duplicates():

    scores[task] = score_set(noholdout[noholdout["task_name"]==task], rectracks)
    
    rprec_sum += scores[task]["rprec"]
    rprec_match += scores[task]["rprec_match"]
    ndcg_sum += scores[task]["ndcg"]
    ndcg_match += scores[task]["ndcg_match"]
    
    
    print("task: {}".format(task))
    print("rprec = {}".format(scores[task]["rprec"]))
    print("ndcg = {}".format(scores[task]["ndcg"]))
    print("rprec_match = {}".format(scores[task]["rprec_match"]))
    print("ndcg_match = {}".format(scores[task]["ndcg_match"]))
    print("\n")

scores["total"] = dict()
scores["total"]["rprec"] = rprec_sum/10
scores["total"]["rprec_match"] = rprec_match
scores["total"]["ndcg"] = ndcg_sum/10
scores["total"]["ndcg_match"] = ndcg_match

print("task: {}".format("total"))
print("rprec = {}".format(scores["total"]["rprec"]))
print("ndcg = {}".format(scores["total"]["ndcg"]))
print("rprec_match = {}".format(scores["total"]["rprec_match"]))
print("ndcg_match = {}".format(scores["total"]["ndcg_match"]))
print("\n")

In [ ]:
noholdout2

In [ ]:
def score_setdf(answer, candidate):

    rprec_sum = 0.0
    ndcg_sum = 0.0
    rprec_match = 0
    ndcg_match = 0

    results = answer.copy()

    #results["candidates"] = 
    results["rprec"] = 0
    #results["rprec_alt"] = 0
    results["ndcg"] = 0

    for pid in results.challenge_pid:
        #pid = startpid + i
        #print("pid={}".format(pid))
        gttracks = [track["track_uri"] for track in results[results["challenge_pid"]==pid].tracks.to_list()[0]]
        candtracks = rectracks.loc[pid].to_list()
        rprec = get_r_precision(gttracks, candtracks)
        #if not results[results["challenge_pid"]==pid].random:
        #    num_samples=results[results["challenge_pid"]==pid].num_samples
        #    rprec_alt =  get_r_precision(gttracks[num_samples:], candtracks)
        rprec_sum = rprec_sum + rprec
        if rprec > 0:
            rprec_match += 1
        ndcg = get_ndcg(gttracks, candtracks)
        ndcg_sum = ndcg_sum + ndcg
        if ndcg > 0:
            ndcg_match += 1

        #results.at[results["challenge_pid"]==pid, "candidates"] = candtracks
        results.at[results["challenge_pid"]==pid, "rprec"] = rprec
        #results.at[results["challenge_pid"]==pid, "rprec_alt"] = rprec_alt
        results.at[results["challenge_pid"]==pid, "ndcg"] = ndcg
        
    return results


submissions[4:6]

noholdout2[noholdout2["challenge_pid"]==0].random

In [ ]:
%%time

rlist=[]
results = pd.DataFrame()
tmpdf = pd.DataFrame()

for submission in submissions:
    print("submission: {}".format(submission))
    print("file: {}".format(submission["file"]))
    
    cachefile=submission["dir"]+"cache/scored-pickle-"+submission["file"]

    start_time = time.time()    
    if (os.path.isfile(cachefile)):
        tmpdf = pd.read_pickle(cachefile)
        print("score read cache: {} sec".format(time.time()-start_time))
    else:
        start_time = time.time()
        rectracks=pd.read_csv('{}/{}'.format(submission["dir"], submission["file"]), header=None, skiprows=1, index_col=0, skipinitialspace=True)
        print("load time: {} sec".format(time.time()-start_time))

        #if ("full" in submission["tag"]):
        if (submission["challenge"]=="mympd-full"):
            print("scored with noholdout2")
            tmpdf = score_setdf(noholdout2, rectracks)
        else:    
            print("scored with noholdout")
            tmpdf = score_setdf(noholdout, rectracks)
        print("score time: {} sec".format(time.time()-start_time))

        tmpdf["trainsizek"]=submission["trainsizek"]

        tmpdf.to_pickle(cachefile)

    print("tmpdf len: {}".format(len(tmpdf)))
    #tmpdf["trainsizek"]=submission["trainsizek"]
    tmpdf["tag"]=submission["tag"]
    tmpdf["method"]=submission["method"]
    rlist.append(tmpdf)
    
results=pd.concat(rlist)
results.reset_index(drop=True, inplace=True)
del rlist

### Summarize total peformance

In [ ]:
results[results["method"]=="mfals"]

In [ ]:
results.groupby(["method", "trainsizek"]).rprec.mean().reindex()

In [ ]:
results.groupby(["tag", "trainsizek"]).ndcg.mean()

In [ ]:
scaleout=results[~results["tag"].isin(["vl6", "hw", "u2uknn", "i2iknn", "mfals"])]

In [ ]:
scaleout

In [ ]:
scaleout.rename(columns={"trainsizek": "trainsize"}, inplace=True)

In [ ]:
scaleout[["method", "trainsize", "ignore"]] = scaleout["tag"].str.split(r"-(.+)k", expand=True)

In [ ]:
scaleout.drop(columns=["ignore"], inplace=True)

In [ ]:
scaleout["trainsize"]=pd.to_numeric(scaleout["trainsize"])

In [ ]:
scaleout[["tag","task_name", "trainsize", "method", "rprec", "ndcg"]]

In [ ]:
means = scaleout.groupby(["tag", "task_name", "trainsize", "method"], sort=False).mean().reset_index()

In [ ]:
scaleout["trainsize"]=pd.to_numeric(scaleout["trainsize"])

In [ ]:
means = scaleout.groupby(["tag", "task_name", "trainsize"], sort=False).mean().reset_index()

In [ ]:
means

In [ ]:
means[(means.method == "mfalsfull") & (means.task_name == "first_5_title")][["method","task_name","trainsize","rprec"]]                                    

In [ ]:
sns.catplot(x = "task_name",       # x variable name
            y = "rprec",       # y variable name
            hue = "tag",  # group variable name
            data = means,     # dataframe to plot
            kind = "bar",
            aspect = 2)
ignore=plt.xticks(rotation=45)
ignore=plt.title("R-Precision Compared")

In [ ]:
sns.catplot(x = "task_name",       # x variable name
            y = "ndcg",       # y variable name
            hue = "tag",  # group variable name
            data = means,     # dataframe to plot
            kind = "bar",
            aspect = 2)
ignore=plt.xticks(rotation=45)
ignore=plt.title("NDCG Compared")

In [ ]:
means[(means["method"]=="u2uknnfull") & (means["task_name"]=="rand_100_title")]

In [ ]:
g = sns.FacetGrid(means, col="task_name", hue="tag", col_wrap=4, sharex=False, margin_titles=True, 
                  height=6, aspect=.75,
                  col_order=["first_5_title", "first_5_wo_title", "first_10_title", "first_10_wo_title",
                            "first_25_title", "first_100_title", "rand_25_title", "rand_100_title",
                            "first_1_title", "title_only"])
g.map(sns.lineplot, "trainsize", "rprec", marker="o", legend="full", linewidth = 4)
g.add_legend()
g.set_titles(col_template="{col_name}")
#g.set(xscale="log")

In [ ]:
g = sns.FacetGrid(means, col="task_name", hue="method", col_wrap=4, sharex=False, margin_titles=True, 
                  height=6, aspect=.75,
                  col_order=["first_5_title", "first_5_wo_title", "first_10_title", "first_10_wo_title",
                            "first_25_title", "first_100_title", "rand_25_title", "rand_100_title",
                            "first_1_title", "title_only"])
g.map(sns.lineplot, "trainsize", "rprec", marker="o", legend="full", linewidth = 4)
g.add_legend()
g.set_titles(col_template="{col_name}")
#g.set(xscale="log")

In [ ]:
sns.catplot(x = "task_name",       # x variable name
            y = "rprec",       # y variable name
            hue = "tag",  # group variable name
            data = means,     # dataframe to plot
            kind = "bar",
            aspect = 2)
ignore=plt.xticks(rotation=45)
ignore=plt.title("R-Precision Compared")

In [ ]:
g = sns.FacetGrid(means, col="task_name", hue="tag", col_wrap=4, sharex=False, margin_titles=True, 
                  height=6, aspect=.75,
                  col_order=["first_5_title", "first_5_wo_title", "first_10_title", "first_10_wo_title",
                            "first_25_title", "first_100_title", "rand_25_title", "rand_100_title",
                            "first_1_title", "title_only"])
g.map(sns.lineplot, "trainsize", "ndcg", marker="o", legend="full", linewidth = 4)
g.add_legend()
g.set_titles(col_template="{col_name}")
#g.set(xscale="log")

Remove rand100 which is the task which most significantly differentiates method performance.

In [ ]:
results[results.task_name!="rand_100_title"].groupby(["tag"]).rprec.mean()

In [ ]:
results[results.task_name!="rand_100_title"].groupby(["tag"]).ndcg.mean()

In [ ]:
means

In [ ]:
g = sns.FacetGrid(results, row="task_name", col="tag", hue="tag", margin_titles=True, height=3, aspect=4)
g.map(sns.violinplot, "tag", "rprec", palette="muted", inner="quart", cut=0, orient='v', scale='count')

In [ ]:
nodup = results[results["tag"]=="vl6"].drop_duplicates(subset="pid")

In [ ]:
nodup = nodup.append(results[results["tag"]=="hw"].drop_duplicates(subset="pid"))

In [ ]:
nodup = nodup.append(results[results["tag"]=="u2uknnfull-20k"].drop_duplicates(subset="pid"))
nodup = nodup.append(results[results["tag"]=="u2uknnfull-40k"].drop_duplicates(subset="pid"))
nodup = nodup.append(results[results["tag"]=="u2uknnfull-60k"].drop_duplicates(subset="pid"))
nodup = nodup.append(results[results["tag"]=="u2uknnfull-80k"].drop_duplicates(subset="pid"))
nodup = nodup.append(results[results["tag"]=="u2uknnfull-100k"].drop_duplicates(subset="pid"))

nodup = nodup.append(results[results["tag"]=="i2iknnfull-20k"].drop_duplicates(subset="pid"))
nodup = nodup.append(results[results["tag"]=="i2iknnfull-40k"].drop_duplicates(subset="pid"))
nodup = nodup.append(results[results["tag"]=="i2iknnfull-60k"].drop_duplicates(subset="pid"))
nodup = nodup.append(results[results["tag"]=="i2iknnfull-80k"].drop_duplicates(subset="pid"))
nodup = nodup.append(results[results["tag"]=="i2iknnfull-100k"].drop_duplicates(subset="pid"))

nodup = nodup.append(results[results["tag"]=="mfalsfull-20k"].drop_duplicates(subset="pid"))
nodup = nodup.append(results[results["tag"]=="mfalsfull-40k"].drop_duplicates(subset="pid"))
nodup = nodup.append(results[results["tag"]=="mfalsfull-60k"].drop_duplicates(subset="pid"))
nodup = nodup.append(results[results["tag"]=="mfalsfull-80k"].drop_duplicates(subset="pid"))
nodup = nodup.append(results[results["tag"]=="mfalsfull-100k"].drop_duplicates(subset="pid"))


In [ ]:
nodup = nodup.append(results[results["tag"]=="i2iknn"].drop_duplicates(subset="pid"))

In [ ]:
nodup = nodup.append(results[results["tag"]=="mfals"].drop_duplicates(subset="pid"))

In [ ]:
nodup.groupby(["tag", "task_name"]).rprec.mean()

In [ ]:
means= nodup.groupby(["tag", "task_name"]).rprec.mean()

In [ ]:

ax=means.vl6.plot.bar(legend=True, color="blue", alpha=.35)
means.hw.plot.bar(ax=ax,legend=True, color="red", alpha=0.35)
means.u2uknn.plot.bar(ax=ax,legend=True, color="purple", alpha=0.35)

Create means as dataframe rather than groupby object by using the aggregator on the groupby object rather than the series of rprec column alone.  This keeps the column names and should simplify plotting.

https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

In [ ]:
means = nodup[["tag", "task_name","rprec"]].groupby(["tag", "task_name"], sort=False).mean().reset_index()

In [ ]:
sns.catplot(x = "task_name",       # x variable name
            y = "rprec",       # y variable name
            hue = "tag",  # group variable name
            data = means,     # dataframe to plot
            kind = "bar",
            aspect = 2)
ignore=plt.xticks(rotation=45)
ignore=plt.title("R-Precision Compared")

In [ ]:
means = nodup[["tag", "task_name","ndcg"]].groupby(["tag", "task_name"], sort=False).mean().reset_index()

In [ ]:
sns.catplot(x = "task_name",       # x variable name
            y = "ndcg",       # y variable name
            hue = "tag",  # group variable name
            data = means,     # dataframe to plot
            kind = "bar",
            aspect = 2)
ignore=plt.xticks(rotation=45)
ignore=plt.title("NDCG Compared")

In [ ]:
scaling=results[results["tag"]in ["v]

In [ ]:
nodup[["tag", "task_name","ndcg"]].groupby(["tag", "task_name"], sort=False).mean().reset_index()

In [ ]:
sns.lineplot(x = "task_name",       # x variable name
            y = "rprec",       # y variable name
            hue = "tag",  # group variable name
            data = means,     # dataframe to plot
            kind = "bar",
            aspect = 2)
ignore=plt.xticks(rotation=45)
ignore=plt.title("R-Precision Compared")

## Explore Violin plot

start with the routine used during the recsys18 analysis

In [ ]:
def plot_violin(df, title="Violin Plot"):
    #sns.set_style("white") 
    sns.set(rc={'figure.figsize':(8,6)})
    g = sns.violinplot(data=df, cut=0, orient='v', scale='width')
    #g = sns.violinplot(x=df.iloc[,0], y=df.iloc[0,:], cut=0, scale='width')
    g.set_title(title)
    #g.set_xlabel("Subtask")
    g.set_ylabel("Score")
    g.set_xticklabels(g.get_xticklabels(), rotation=90)

    return g

In [ ]:
nodup.groupby(["tag", "task_name"]).rprec.describe().T

Don't need the describe data because that will come from the violin plot.  Just need to use all the raw data points.

In [ ]:
vl6desc = nodup[nodup["tag"]=="vl6"][["task_name", "rprec"]] #.groupby(["task_name"]).mean()

In [ ]:
hwdesc = nodup[nodup["tag"]=="hw"][["task_name", "rprec"]]

In [ ]:
plot_violin(vl6desc[vl6desc["task_name"]=="rand_100_title"], "rand_100_title")

In [ ]:
plot_violin(hwdesc[hwdesc["task_name"]=="rand_100_title"], "rand_100_title")

## Explore side-by-side plots

What to take all the tasks and see the teams side by side

Create violin plots for each task. https://stackoverflow.com/a/47487445/8928529

Basically loop through the tasks and plot on each subplot axis.

In [ ]:
fig, axes = plt.subplots(10, 2, figsize=(20, 32), sharey='row')
axes_cols = (axes.flatten()[::2], axes.flatten()[1::2])

i=0
for task in vl6desc.task_name.drop_duplicates():
    
    ax=axes_cols[0][i]
    sns.violinplot(data=vl6desc[vl6desc["task_name"]==task], cut=0, orient='v', scale='width', ax=ax)
    ax.set_title('task = {}'.format(task), y=0.95)
    ax=axes_cols[1][i]
    sns.violinplot(data=hwdesc[hwdesc["task_name"]==task], cut=0, orient='v', scale='width', ax=ax)
    ax.set_title('task = {}'.format(task), y=0.95)
    
    
    i += 1

## Explore split plot

This lets me see the data paired directly and allows easier visual comparison of differences.

The test rand_100_title task shows clear differences.

In [ ]:
data = nodup[((nodup["tag"]=="vl6") | (nodup["tag"]=="hw"))][["tag","task_name", "rprec"]]

In [ ]:
sns.violinplot(data=data[data["task_name"]=="rand_100_title"], cut=0, orient='v', scale='width',
               x="task_name", y="rprec",
               hue="tag",
               split=True, inner="quart")

#sns.despine(left=True)

In [ ]:
rprec_data = nodup[((nodup["tag"]=="vl6") | (nodup["tag"]=="hw"))][["tag","task_name", "rprec"]]
ndcg_data  = nodup[((nodup["tag"]=="vl6") | (nodup["tag"]=="hw"))][["tag","task_name", "ndcg"]]

In [ ]:
fig, axes = plt.subplots(10, 2, figsize=(20, 32), sharey='row')
axes_cols = (axes.flatten()[::2], axes.flatten()[1::2])

i=0
for task in vl6desc.task_name.drop_duplicates():
    
    ax=axes_cols[0][i]
    sns.violinplot(data=rprec_data[rprec_data["task_name"]==task], cut=0, orient='v', scale='width',
               x="task_name", y="rprec",
               hue="tag",
               split=True, inner="quart", ax=ax)
    ax.set_title('task = {}'.format(task), y=0.95)

    ax=axes_cols[1][i]
    sns.violinplot(data=ndcg_data[ndcg_data["task_name"]==task], cut=0, orient='v', scale='width',
               x="task_name", y="ndcg",
               hue="tag",
               split=True, inner="quart", ax=ax)
    ax.set_title('task = {}'.format(task), y=0.95)
    
    
    i += 1

With the rprec and ndcg data plotted side by side it is clear that the rand_100_title was much more effectively solved by vl6.

The second most effective was first_1_title where vl6 had higher mean.

However the the title_only solution of hw is clearly better.

All the rest of the tasks had nearly identical means and distributions with the hw solution having slightly higher ndcg in those tasks.


In [ ]:
task

In [ ]:
rprec_data = nodup[["tag","task_name", "rprec"]]
ndcg_data  = nodup[["tag","task_name", "ndcg"]]

In [ ]:
rprec_data=rprec_data.reset_index().rename(columns={"index": "pid"})

In [ ]:
rprec_data[((rprec_data["tag"]=="vl6") & (rprec_data["task_name"]=="first_1_title"))]

In [ ]:
sns.scatterplot(data=rprec_data[((rprec_data["tag"]=="vl6") & (rprec_data["task_name"]=="first_1_title"))], x='pid', y="rprec")

In [ ]:
sns.scatterplot(data=rprec_data[((rprec_data["tag"]=="hw") & (rprec_data["task_name"]=="first_1_title"))], x='pid', y="rprec")

In [ ]:
sns.scatterplot(data=rprec_data[((rprec_data["tag"]=="vl6"))], x="pid", y="rprec", hue="tag", alpha=.5)
#sns.scatterplot(data=rprec_data[((rprec_data["tag"]=="hw"))], y="rprec", hue="tag", alpha=.5, )

In [ ]:
sns.scatterplot(data=rprec_data, x="pid", y="rprec", hue="tag", alpha=0.4)

It's much easier to get fast insight from seaborn using facet grids since they are built right from the data.

https://seaborn.pydata.org/tutorial/axis_grids.html

In [ ]:
g = sns.FacetGrid(rprec_data, row="task_name", col="tag", hue="tag", margin_titles=True, height=4, aspect=2)
g.map(sns.scatterplot, "pid", "rprec", linewidth = 0)

In [ ]:
rprec_data["rpid"]= rprec_data.pid % 1000

In [ ]:
g = sns.FacetGrid(rprec_data, row="task_name", col="tag", hue="tag", margin_titles=True, height=3, aspect=2)
g.map(sns.scatterplot, "rpid", "rprec", linewidth = 0)

Revisit violin distribution plots now with all data.

In [ ]:
g = sns.FacetGrid(rprec_data, row="task_name", hue="tag", margin_titles=True, height=3, aspect=4)
g.map(sns.violinplot, "tag", "rprec", order=['vl6', 'hw', 'u2uknn', 'u2uknnfull', 'u2uknnfull-10k', 'u2uknnfull-20k', 'u2uknnfull-30k', 'u2uknnfull-40k', 'u2uknnfull-50k', 'u2uknnfull-60k', 'u2uknnfull-70k', 'u2uknnfull-80k', 'u2uknnfull-90k', 'u2uknnfull-100k', 'i2iknn','mfals'], palette="muted", inner="quart", cut=0, orient='v', scale='count')

##  Inspect the highest scoring recommendations

Understand which tasks and methods perform the best.

In [ ]:
nodup[nodup.rprec > 0.8][["rprec", "ndcg", "tag","task_name", "name", "num_tracks"]]